https://www.youtube.com/watch?v=ftMq5ps503w

In [1]:
import pickle

import numpy as np
import pandas as pd

import tensorflow as tf

https://www.tensorflow.org/programmers_guide/variables

In [2]:
X = pd.read_csv('sp500.csv')

In [3]:
X.shape

(4170, 1)

In [4]:
def normalise_windows(window_data):
    normalised_data = []
    for window in window_data:
        normalised_window = [((float(p) / float(window[0])) - 1) for p in window]
        normalised_data.append(normalised_window)
    return normalised_data

In [5]:
def load_data(filename, seq_len, normalise_window):
    f = open(filename, 'r').read()
    data = f.split('\n')

    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])
    
    if normalise_window:
        result = normalise_windows(result)

    result = np.array(result)
    
    row = round(0.9 * result.shape[0])
    train = result[:int(row), :]
    np.random.shuffle(train)
    x_train = train[:, :-1]
    y_train = train[:, -1]
    x_test = result[int(row):, :-1]
    y_test = result[int(row):, -1]

    #x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    #x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))  
    #x_train = x_train.transpose()
    #x_test = y_test.transpose()
    return [x_train, y_train, x_test, y_test]

In [6]:
batch_size = 128
lstm_units = 100
seq_len = 50

X_train, y_train, X_test, y_test = load_data('sp500.csv', seq_len, True)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((3709, 50), (3709,), (412, 50), (412,))

In [7]:
def get_batch(batch_size=64, X=X_train, y=y_train):
    n_batches = X.shape[0]//batch_size
    for i in range(n_batches):
        yield X[i*batch_size:(i+1)*batch_size, :].reshape(batch_size, -1, 1), y[i*batch_size : (i+1)*batch_size]

In [8]:

X_test.reshape((X_test.shape[0], X_test.shape[1], 1)).shape

(412, 50, 1)

In [9]:
inputs = tf.placeholder(tf.float32, shape = [None, 50, 1], name='inputs')
outputs = tf.placeholder(tf.float32, shape = [batch_size])
learining_rate = tf.placeholder(tf.float32)

In [10]:
with tf.name_scope('rec_layer'):
    rnn_cell_1 = tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(num_units = lstm_units)
    #initial_state = rnn_cell_1.zero_state(batch_size=batch_size, dtype=tf.float32)
    lstm_output, lstm_state = tf.nn.dynamic_rnn(
        cell=rnn_cell_1, 
        inputs=inputs, dtype=tf.float32)

In [11]:
with tf.name_scope('output_layer'):
    prediction = tf.contrib.layers.fully_connected(inputs=lstm_output[-1], num_outputs=1, activation_fn=None)
    predictions = tf.identity(prediction, name="predictions")
    loss = tf.reduce_sum(tf.square(predictions-outputs))

In [12]:
train = tf.train.AdamOptimizer(learning_rate=learining_rate).minimize(loss)

In [13]:
LEARMING_RATE = 0.05
epochs = 5
save_dir = './save55'

feed_dict = {learining_rate: LEARMING_RATE}

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    feeed_dict = {learining_rate: LEARMING_RATE}
    for epoch in range(epochs):
        batches = get_batch(batch_size)
        for batch in batches:
            feed_dict[inputs], feed_dict[outputs] = batch
            sess.run(train, feed_dict)
        print("Epoch: {}, loss: {}".format(epoch, sess.run(loss, feed_dict)))
    out, st = sess.run([lstm_output, lstm_state], feed_dict)
    print(out[-1])
    print(st)
    feed_dict[inputs] = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    out, st = sess.run([lstm_output, lstm_state], feed_dict)
    print(out[-1])
    print(st)
    saver = tf.train.Saver()
    saver.save(sess, save_dir)

Epoch: 0, loss: 49.05851364135742
Epoch: 1, loss: 34.999900817871094
Epoch: 2, loss: 35.517433166503906
Epoch: 3, loss: 35.12543869018555
Epoch: 4, loss: 35.100982666015625
[[  5.73048294e-02   1.12502195e-03  -1.55451950e-02 ...,  -2.81887036e-02
    4.34004813e-02  -6.94672093e-02]
 [  2.12794058e-02   3.59711200e-02  -2.35688627e-01 ...,   9.17696021e-03
    5.27546601e-03  -1.47115722e-01]
 [  2.58257557e-02   2.07414571e-02  -3.56966794e-01 ...,   4.68126778e-03
   -4.66544135e-03  -1.24071971e-01]
 ..., 
 [  5.61492369e-02   6.00541895e-03  -4.42370743e-01 ...,   1.57618357e-04
   -6.41458342e-03  -1.25730932e-01]
 [  5.65473661e-02   5.98025415e-03  -4.42483366e-01 ...,   1.61943695e-04
   -6.42663613e-03  -1.25549585e-01]
 [  5.68990782e-02   5.98569913e-03  -4.41014111e-01 ...,   1.51344444e-04
   -6.43238658e-03  -1.25696152e-01]]
LSTMStateTuple(c=array([[  1.79989457e-01,   8.84241387e-02,  -5.77369571e-01, ...,
          3.09018744e-03,  -7.61723667e-02,  -4.61305857e+00],


In [14]:
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    feed_dict = {}
    loader = tf.train.import_meta_graph(save_dir + '.meta')
    loader.restore(sess, save_dir)
    variables_names = [v.name for v in tf.trainable_variables()]
    inputs = loaded_graph.get_tensor_by_name("inputs:0")
    feed_dict[inputs] = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    print(variables_names)

INFO:tensorflow:Restoring parameters from ./save55
['rnn/basic_lstm_cell/weights:0', 'rnn/basic_lstm_cell/biases:0', 'fully_connected/weights:0', 'fully_connected/biases:0']
